Kristupas Norvaiša, 2016012, II grupė

Imports

In [2]:
!pip install openimages
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from openimages.download import download_dataset
import os
if not torch.cuda.is_available():
    device = torch.device("cpu")
else:
    device = torch.device("cuda")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 10.5 MB/s eta 0:00:00


Defining convolutional network

In [3]:
class SimpleConvNet(nn.Module):
    def __init__(self, in_shape, out_classes):
        super().__init__()
        self.num_classes = out_classes
        self.conv1_1 = nn.Conv2d(in_shape[0], 8, (3, 3), padding = 'same')
        self.conv1_2 = nn.Conv2d(8, 8, (3, 3), padding = 'same')
        self.conv2_1 = nn.Conv2d(8, 16, (3, 3), padding = 'same')
        self.conv2_2 = nn.Conv2d(16, 16, (3, 3), padding = 'same')
        self.fc1 = nn.Linear(16 * (in_shape[1] // 4) * (in_shape[2] // 4), 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, out_classes)

    def forward(self, x):
        y = nn.Sequential(
            self.conv1_1,
            nn.ReLU(),
            self.conv1_2,
            nn.ReLU(),
            nn.MaxPool2d((2, 2), (2, 2)),
            self.conv2_1,
            nn.ReLU(),
            self.conv2_2,
            nn.ReLU(),
            nn.MaxPool2d((2, 2), (2, 2)),
            nn.Flatten(),
            self.fc1,
            nn.ReLU(),
            self.fc2,
            nn.ReLU(),
            self.fc3
        )(x)
        return y

Training and evaluation of the model, data preprocessing, metrics calculation

In [9]:
def train_and_eval(convnet, dl_train, dl_valid, n_epochs, criterion, optimizer):
    for epoch in range(n_epochs):
        convnet.train()
        for images, labels in dl_train:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            predictions = convnet(images)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()

        convnet.eval()
        true_labels = []
        pred_labels = []
        for images, labels in dl_valid:
            images, labels = images.to(device), labels.to(device)
            predictions = convnet(images)
            _, preds = torch.max(predictions, 1)
            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(preds.cpu().numpy())

        acc = accuracy_score(true_labels, pred_labels)
        prec, rec, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average=None)
        cm = confusion_matrix(true_labels, pred_labels)
        class_acc = cm.diagonal() / cm.sum(axis=1)
        class_names = ["Bee", "Goldfish", "Goose"]
        
        print(f"Epoch {epoch+1}/{n_epochs} -- Accuracy: {acc:.3f} -- Precision: {prec} -- Recall: {rec} -- F1: {f1} -- Confusion Matrix:\n{cm}")
        for i, class_name in enumerate(class_names):
            print(f"Accuracy for {class_name}: {class_acc[i]:.3f}")

transform_train = T.Compose([
    T.RandomRotation(10),
    T.Resize((64, 64)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_valid = T.Compose([
    T.Resize((64, 64)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

if not os.path.exists("images"):
    download_dataset("images", ["Bee", "Goldfish", "Goose"], annotation_format=None, limit=10)

train_data = datasets.ImageFolder('/content/images', transform=transform_train)
valid_data = datasets.ImageFolder('/content/images', transform=transform_valid)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=False)

input_shape = (3, 64, 64)
output_classes = 3
convnet = SimpleConvNet(input_shape, output_classes).to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(convnet.parameters(), lr=0.001)

epochs = 10
train_and_eval(convnet, train_loader, valid_loader, epochs, loss_func, optimizer)

Epoch 1/10 -- Accuracy: 0.333 -- Precision: [0.33333333 0.         0.        ] -- Recall: [1. 0. 0.] -- F1: [0.5 0.  0. ] -- Confusion Matrix:
[[10  0  0]
 [10  0  0]
 [10  0  0]]
Accuracy for Bee: 1.000
Accuracy for Goldfish: 0.000
Accuracy for Goose: 0.000
Epoch 2/10 -- Accuracy: 0.367 -- Precision: [1.         0.34482759 0.        ] -- Recall: [0.1 1.  0. ] -- F1: [0.18181818 0.51282051 0.        ] -- Confusion Matrix:
[[ 1  9  0]
 [ 0 10  0]
 [ 0 10  0]]
Accuracy for Bee: 0.100
Accuracy for Goldfish: 1.000
Accuracy for Goose: 0.000
Epoch 3/10 -- Accuracy: 0.433 -- Precision: [0.    0.5   0.375] -- Recall: [0.  0.7 0.6] -- F1: [0.         0.58333333 0.46153846] -- Confusion Matrix:
[[0 3 7]
 [0 7 3]
 [0 4 6]]
Accuracy for Bee: 0.000
Accuracy for Goldfish: 0.700
Accuracy for Goose: 0.600
Epoch 4/10 -- Accuracy: 0.333 -- Precision: [0.         0.         0.33333333] -- Recall: [0. 0. 1.] -- F1: [0.  0.  0.5] -- Confusion Matrix:
[[ 0  0 10]
 [ 0  0 10]
 [ 0  0 10]]
Accuracy for Bee: 0

Image input to label output

In [5]:
def predict_picture(img):
    convnet.eval()

    with torch.no_grad():
        pred = convnet(transform_valid(img).unsqueeze(0).to(device))
    label_pred = torch.argmax(pred, axis=1)

    classes = ["Bee", "Goldfish", "Goose"]
    return classes[label_pred]

Web application

In [6]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

!./ngrok authtoken 2NvAbgVgYtGDBWBVC1xleWHamBO_234SofNE5TzUMKMMBGyx7

from flask import Flask, render_template, request
from PIL import Image as im
import cv2

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok

app = Flask(__name__, template_folder='/content')
run_with_ngrok(app)

Completed!
--2023-04-12 06:52:33--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  3.35MB/s    in 6.9s    

2023-04-12 06:52:40 (1.91 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [7]:
@app.route("/")
def hello():
    return render_template('index.html')

In [8]:
@app.route('/', methods=['POST'])
def upload_image():
    filestr = request.files['file'].read()
    file_bytes = np.fromstring(filestr, np.uint8)
    img_array = cv2.imdecode(file_bytes, cv2.IMREAD_UNCHANGED)
    img = im.fromarray(img_array)

    return f"<h1>The picture is probably: {predict_picture(img)}</h1>"

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f57f-35-204-211-180.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:53:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:53:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:54:08] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:55:27] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:55:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:55:46] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:57:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:57:13] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:57:43] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:57:48] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:57:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:58:24] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Apr/2023 06:59:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0